In [8]:
import pandas as pd
import seaborn as sb

In [9]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
time_now="Updated Time "+ current_time

In [4]:
print (time_now)

Updated Time 22:08:13


In [10]:
malaysia_covid_url='https://query.data.world/s/v55vhdiey6gg5rq6wacpo3u4umsk65'
malaysia_vaccine_url='https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_malaysia.csv'
df_my_covid = pd.read_csv(malaysia_covid_url)
df_my_vacci = pd.read_csv(malaysia_vaccine_url)

In [5]:
df_my_covid

,date,location,new_cases,new_deaths,total_cases,total_deaths,recover,total_recover,icu,support
0,2019-12-31,Malaysia,0,0,0,0,0,0,0,0
1,2020-01-01,Malaysia,0,0,0,0,0,0,0,0
2,2020-01-02,Malaysia,0,0,0,0,0,0,0,0
3,2020-01-03,Malaysia,0,0,0,0,0,0,0,0
4,2020-01-04,Malaysia,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
544,2021-06-27,Malaysia,5586,60,734048,4944,4777,667709,886,446
545,2021-06-28,Malaysia,5218,57,739266,5001,4744,672453,899,451
546,2021-06-29,Malaysia,6437,107,745703,5108,5298,677751,905,455
547,2021-06-30,Malaysia,6276,62,751979,5170,4929,682680,905,452


In [11]:
df_cov_clean=df_my_covid[["date","new_cases","icu"]].set_index("date")
df_vacc_clean=df_my_vacci[["date","dose2_cumul"]].set_index("date")
df_comb=df_vacc_clean.join(df_cov_clean)
df_comb=df_comb.dropna()
data_analytic=df_comb.tail(7)
table=df_comb.tail(7).to_html().replace('<table border="1" class="dataframe">','<table class="table">')

In [12]:
data_analytic.tail().to_html().replace('<table border="1" class="dataframe">','<table class="table">')

'<table class="table">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>dose2_cumul</th>\n      <th>new_cases</th>\n      <th>icu</th>\n    </tr>\n    <tr>\n      <th>date</th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>2021-06-27</th>\n      <td>2087056</td>\n      <td>5586</td>\n      <td>886</td>\n    </tr>\n    <tr>\n      <th>2021-06-28</th>\n      <td>2130051</td>\n      <td>5218</td>\n      <td>899</td>\n    </tr>\n    <tr>\n      <th>2021-06-29</th>\n      <td>2214001</td>\n      <td>6437</td>\n      <td>905</td>\n    </tr>\n    <tr>\n      <th>2021-06-30</th>\n      <td>2309018</td>\n      <td>6276</td>\n      <td>905</td>\n    </tr>\n    <tr>\n      <th>2021-07-01</th>\n      <td>2393882</td>\n      <td>6988</td>\n      <td>917</td>\n    </tr>\n  </tbody>\n</table>'

In [53]:
round(data_analytic.new_cases.mean())

5853

In [82]:
date=data_analytic.tail(1).index.values[0]

'2021-06-30'

In [74]:
fully_vacc=round(data_analytic.dose2_cumul.tail(1).mean())
population=32365998
cases_7days_mean=round(data_analytic.new_cases.mean())
icu_7days_mean=round(data_analytic.icu.mean())

date=" updated at "+data_analytic.tail(1).index.values[0] +" on " +time_now

def results_covid(case,case_max,icu,icu_max,vacc):
    pstart="""<p class="lead text-white-50 font-weight-normal">"""
    pend="""</p>"""
    output=""

    if (cases_7days_mean-case_max>0):
        #print("We need to lessen average positive case by :" ,str(int(case-case_max)))
        output=output+pstart+"The Average 7 day case is : <b>"+str(int(cases_7days_mean))+"</b>"+\
        "<br> Our target is  : <b>" +str(int(case_max))+"</b>"+\
        "<br> We need to lessen average positive case by : <b>" +str(int(case-case_max))+"</b>"+pend

    else:
        #print("We Reached The Target")
        output=output+pstart+("<b>We Reached The Target.</b>"+pend)
    if (icu_7days_mean-icu_max>0):
        #print("We need to lessen average ICU case by about :" ,str(int(icu-icu_max)))
        output=output+pstart+("We need to lessen average ICU case by about : <b>" +str(int(icu-icu_max)))+"</b>"+pend
    else:
        #print("ICU seem to be in Acceptable Condition")
        output=output+pstart+"ICU seem to be in <b> Acceptable Condition  The average 7 days ICU cases is : "+ str(int(icu)) +" </b>"+pend
    #print("Our current Vaccination is about: ",str(round(vacc/population*100,2))," of the population")
    output=output+pstart+("Our current Vaccination is about: <b>"+str(round(vacc/population*100,2))+" </b>of the population")+pend
    return output

def header(phase):
    return date+"""<h1 class="display-4 font-weight-normal">"""+phase+"""</h1>"""
    

if ((cases_7days_mean<500)&((icu_7days_mean)<500)&((fully_vacc/population)>.6)):
    #print("We are in Phase 4")
    phase=header("We are in Phase 4")
    phase_text="We are in Phase 4"

elif ((cases_7days_mean<2000)&((icu_7days_mean)<700)&((fully_vacc/population)>.4)):
    #print("We are in Phase 3")
    phase=header("We are in Phase 3")
    phase_text="We are in Phase 3"
    result=results_covid(cases_7days_mean,500,icu_7days_mean,500,fully_vacc)
    #print(result)
elif ((cases_7days_mean<4000)&((icu_7days_mean)<900)&((fully_vacc/population)>.1)):
    #print("We are in Phase 2")
    phase=header("We are in Phase 2")
    phase_text="We are in Phase 2"
    result=results_covid(cases_7days_mean,2000,icu_7days_mean,700,fully_vacc)
    #print(result)
else:
    #print("We are in Phase 1")
    phase=header("We are in Phase 1")
    phase_text="We are in Phase 1"
    result=results_covid(cases_7days_mean,4000,icu_7days_mean,900,fully_vacc)
    #print(result)
result

'<p class="lead text-white-50 font-weight-normal">The Average 7 day case is : <b>6017</b><br> Our target is  : <b>4000</b><br> We need to lessen average positive case by : <b>2017</b></p><p class="lead text-white-50 font-weight-normal">ICU seem to be in <b> Acceptable Condition  The average 7 days ICU cases is : 893 </b></p><p class="lead text-white-50 font-weight-normal">Our current Vaccination is about: <b>7.4 </b>of the population</p>'

'<p class="lead font-weight-normal">The Average 7 day case is : <b>6017</b><br> Our target is  : <b>4000</b><br> We need to lessen average positive case by : <b>2017</b></p><p class="lead font-weight-normal">We need to lessen average ICU case by about : <b>100</b></p><p class="lead font-weight-normal">Our current Vaccination is about: <b>7.4 </b>of the population</p>'

In [76]:
template = open('template.html', 'r')
template_lines = template.readlines()
 
count = 0
# Strips the newline character
final=""
for line in template_lines:
    count += 1
    final=final+line
    #final=line.replace("<$PHASE$>",phase).replace("<$OUTPUT$>",result).replace("<$DATE$>",date).replace("<$TABLE$>",table)
    #final=final+line.replace("<$OUTPUT$>",result)
    #final=final+line.replace("<$PHASE$>",phase)
    #final=final+line.replace("<$DATE$>",date)
    #final=final+line.replace("<$TABLE$>",table)

final=final.replace("<$PHASE$>",phase_text)
final=final.replace("<$DATE$>",date)
final=final.replace("<$OUTPUT$>",result)
final=final.replace("<$TABLE$>",table)
f = open("index.html", "w")
f.write(final)
f.close()



In [58]:
print(final)

<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
        <meta name="description" content="" />
        <meta name="author" content="" />
        <title>Are We There Yet Malaysia MCO 3.0 </title>
        <!-- Favicon-->
        <link rel="icon" type="image/x-icon" href="assets/favicon.ico" />
        <!-- Bootstrap icons-->
        <link href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.4.1/font/bootstrap-icons.css" rel="stylesheet" />
        <!-- Core theme CSS (includes Bootstrap)-->
        <link href="css/styles.css" rel="stylesheet" />
    </head>
    <body>
        <!-- Responsive navbar-->
        <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
            <div class="container px-5">
                <a class="navbar-brand" href="#!">Are We There Yet Malaysia MCO 3.0</a>
                <button class="navbar-toggler" type="button" data-bs-tog

In [59]:
f = open("index.html", "w")
f.write(final)
f.close()
